In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Definimos fuente para los graficos
from matplotlib import rcParams
import matplotlib.font_manager as font_manager

font_dir = ['/home/daniufundar/Downloads/Archivo/']
for font in font_manager.findSystemFonts(font_dir):
    font_manager.fontManager.addfont(font)

rcParams['font.family'] = 'Archivo'

fundar_colors = ['#E27124','#AFA36E', '#5D896F','#7BB5C4', '#B3B3B3']

In [2]:
import platform

if platform.system() == "Windows":
    pathfolder = 'path_for_Windows'
elif platform.system() == "Linux":
    if platform.uname()[2] == "5.15.0-57-generic":
        pathfolder = '/home/daniufundar/Documents/Fundar/indice-desigualdad-genero-fundar/'
    elif platform.uname()[2] == "version2":
        pathfolder = '/Users/danielarisaro/Documents/fundar/indice-desigualdad-genero/'
elif platform.system() == "Darwin":
    pathfolder = '/Users/danielarisaro/Documents/fundar/indice-desigualdad-genero/'
else:
    raise Exception("Unsupported operating system")


In [3]:
pathfolder = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'
pathinput = pathfolder + 'data_input/'

filename_hogar = 'hogar_T'
filename_personas = 'personas_T'

list_anios = [20, 21]
list_trimestres = [1, 2, 3, 4]

In [26]:
for i, it in enumerate(list_trimestres):
    for j, janios in enumerate(list_anios):

        filename = filename_personas + str(it) + str(janios) + '.txt'
        print(filename)
        df = pd.read_csv(pathinput + filename, delimiter=';', low_memory=False)

personas_T120.txt
personas_T121.txt
personas_T220.txt
personas_T221.txt
personas_T320.txt
personas_T321.txt
personas_T420.txt
personas_T421.txt


In [15]:
#cod_aglomerado

dict_cod_aglomerado = {2: "Gran La Plata",
3: "Bahía Blanca - Cerri",
4: "Gran Rosario",
5: "Gran Santa Fé",
6: "Gran Paraná",
7: "Posadas",
8: "Gran Resistencia",
9: "Comodoro Rivadavia - Rada Tilly",
10:" Gran Mendoza",
12: "Corrientes",
13: "Gran Córdoba",
14: "Concordia",
15: "Formosa",
17: "Neuquén – Plottier",
18: "Santiago del Estero - La Banda",
19: "Jujuy-Palpalá",
20: "Río Gallegos",
22: "Gran Catamarca",
23: "Gran Salta",
25: "La Rioja",
26: "Gran San Luis",
27: "Gran San Juan",
29: "Gran Tucumán - Tafí Viejo",
30: "Santa Rosa – Toay",
31: "Ushuaia - Río Grande",
32: "Ciudad Autónoma de Buenos Aires",
33: "Partidos del GBA",
34: "Mar del Plata",
36: "Río Cuarto",
38: "San Nicolás – Villa Constitución",
91: "Rawson – Trelew",
93: "Viedma – Carmen de Patagones"}

In [ ]:

archivos = ['personas_T120.txt', 'personas_T220.txt', 
            'personas_T320.txt', 'personas_T420.txt', 
            'personas_T121.txt', 'personas_T221.txt', 
            'personas_T321.txt', 'personas_T421.txt']


for archivo in archivos:
    print(archivo)
    
    df = pd.read_csv(pathinput + archivo, delimiter=';', low_memory=False)
    trimestre = archivo.split('T')[1][0]
    
    tipo = 'Aglomerado' 
    dict_cod = dict_cod_aglomerado
    base = 'Individual'
    
    ratio_empleo, error_empleo = ratio_tasa_empleo(df, tipo=tipo, base=base)
    ratio_desempleo, error_desempleo = ratio_tasa_desempleo(df, tipo=tipo, base=base)
    ratio_informalidad, error_informalidad = ratio_tasa_registro(df, tipo=tipo, base=base)
    ratio_horas_remuneradas, error_horas_remuneradas = ratio_tasa_horas_remuneradas(df, tipo=tipo, base=base)
    ratio_poblacion_dependiente, error_poblacion_dependiente = ratio_tasa_poblacion_dependiente(df, tipo=tipo, base=base)
    ratio_ingreso_laboral, error_ingreso_laboral = ratio_ingreso_laboral_mensual(df, tipo=tipo, base=base)
    ratio_pobreza, error_pobreza = ratio_ingreso_laboral_mensual(df, tipo=tipo, base=base)
    
    # Combinar los ratios en un solo dataframe
    df_ratio = df_merge_ratios(ratio_empleo, ratio_desempleo, ratio_informalidad, ratio_horas_remuneradas, 
                               ratio_poblacion_dependiente, ratio_ingreso_laboral, ratio_pobreza, dict_cod_aglomerado)
    

    xlabel = archivo.split('T')[1][0:3]


    # Define the list of indicators to plot
    indicators = ['Actividad productiva', 'Búsqueda de trabajo', 'Acceso a derechos laborales', 
                'Participación en el trabajo remunerado', 'Dependencia económica', \
                    #'Ingreso laboral', 'Pobreza'\
                        ]

    # Define the color palette for the lines
    colors = fundar_colors

    # Create a figure with subplots for each indicator
    fig, axes = plt.subplots(nrows=len(indicators), ncols=1, figsize=(18, 5*len(indicators)))

    # Iterate over each indicator and plot it in a separate subplot
    # Loop through indicators and plot
    for i, indicator in enumerate(indicators):
        ax = axes[i]
        ax.bar(df_ratio.index, df_ratio[indicator], color=colors[i], alpha=0.7)
        ax.set_title(indicator)
        ax.set_xlabel(xlabel)
        ax.set_ylabel('Ratio')
        # get the current xticks
        xticks = ax.get_xticks()

        # set new xtick labels
        new_xtick_labels = df_ratio.index.tolist()
        ax.set_xticklabels(new_xtick_labels)

        # rotate the xtick labels
        ax.tick_params(axis='x', rotation=90)

    # Adjust the spacing between subplots
    plt.subplots_adjust(hspace=0.5)

    # Show the plot
    #plt.show()
    fig.savefig(pathfolder + xlabel + '.png', dpi=300, bbox_inches = 'tight')


In [78]:
archivos = ['personas_tot_urb_3T_20.txt', 'personas_tot_urb_3T_21.txt']

for archivo in archivos:
    print(archivo)
    
    df = pd.read_csv(pathinput + archivo, delimiter=';', low_memory=False)
    trimestre = archivo.split('T')[1][0]
    
    tipo =  'Urbano'
    dict_cod = dict_cod_provincia
    base = 'Individual'
    
    ratio_empleo, error_empleo = ratio_tasa_empleo(df, tipo=tipo, base=base)
    ratio_desempleo, error_desempleo = ratio_tasa_desempleo(df, tipo=tipo, base=base)
    ratio_informalidad, error_informalidad = ratio_tasa_registro(df, tipo=tipo, base=base)
    ratio_horas_remuneradas, error_horas_remuneradas = ratio_tasa_horas_remuneradas(df, tipo=tipo, base=base)
    ratio_poblacion_dependiente, error_poblacion_dependiente = ratio_tasa_poblacion_dependiente(df, tipo=tipo, base=base)
    ratio_ingreso_laboral, error_ingreso_laboral = ratio_ingreso_laboral_mensual(df, tipo=tipo, base=base)
    ratio_pobreza, error_pobreza = ratio_ingreso_laboral_mensual(df, tipo=tipo, base=base)
    
    # Combinar los ratios en un solo dataframe
    df_ratio = df_merge_ratios(ratio_empleo, ratio_desempleo, ratio_informalidad, ratio_horas_remuneradas, 
                               ratio_poblacion_dependiente, ratio_ingreso_laboral, ratio_pobreza, dict_cod_aglomerado)
    


In [ ]:
path_basemaps = '/home/daniu/Documentos/fundar/indice-desigualdad-genero/basemaps/aglomerados_eph/'
path_basemaps = '/home/daniufundar/Documents/Fundar/indice-desigualdad-genero-fundar/basemaps/aglomerados_eph/'

gdf_aglomerados = gpd.read_file(path_basemaps + 'gdf_aglomerados_con_codigo.shp')
gdf_aglomerados['eph_codagl'] = gdf_aglomerados['eph_codagl'].astype(int)
gdf_aglomerados['index'] = gdf_aglomerados['eph_codagl'].map(dict_cod_aglomerado)

gdf_aglomerados.set_index(gdf_aglomerados['index'], inplace=True)

In [ ]:
df = pd.merge(gdf_aglomerados, df_ratio, left_index=True, right_index=True, how='inner')